## Overview

We run coverage simulations for various basic models and proposed non-informative priors.

## Import Depedencies

In [1]:
import numpy as np
import scipy.stats
from scipy.special import gammaincc

## Example 1: Gaussian With Unknown Mean and Known Variance

In [2]:
def generate_data(mean, n):
    return np.random.normal(mean, 1.0, size=n)

def post(y):
    n = len(y)
    m = np.mean(y)
    def f(t):
        return scipy.stats.norm(loc=m, scale=1.0 / n**.5).cdf(t)
    return f

def run_sim(post, N, mean, n):
    cnt = 0
    low = 0.025
    high = 1 - low
    for _ in range(N):
        y = generate_data(mean, n)
        cdf = post(y)
        t = cdf(mean)
        if low < t and t < high:
            cnt += 1
    return cnt / N
print('')
N = 10000
np.random.seed(0)
means = [0.1, 0.5, 1.0, 2.0, 5.0]
res = [[] for _ in means]
print('*** constant prior')
for n in [5, 10, 15, 20]:
    print('** n =', n)
    for i, mean in enumerate(means):
        res[i].append(mean)
        cov = run_sim(post, N, mean, n)
        print(mean, cov)


*** constant prior
** n = 5
0.1 0.9502
0.5 0.9519
1.0 0.9516
2.0 0.9514
5.0 0.9489
** n = 10
0.1 0.9486
0.5 0.9478
1.0 0.9495
2.0 0.9521
5.0 0.9455
** n = 15
0.1 0.9508
0.5 0.9492
1.0 0.9517
2.0 0.9539
5.0 0.9558
** n = 20
0.1 0.9493
0.5 0.9488
1.0 0.9494
2.0 0.9489
5.0 0.9488


## Example 2: Gaussian With Known Mean and Unknown Variance

In [3]:
def generate_data(sigma2, n):
    return np.random.normal(0, sigma2**.5, size=n)

def post1(y):
    n = len(y)
    s2 = np.dot(y, y) / n
    def f(t):
        if t == 0:
            return 0
        return 1 - gammaincc((n-2)/2, n * s2 / (2 * t))
    return f

def post2(y):
    n = len(y)
    s2 = np.dot(y, y) / n
    def f(t):
        if t == 0:
            return 0
        return 1 - gammaincc(n/2, n * s2 / (2 * t))
    return f

def run_sim(post, N, sigma2, n):
    cnt = 0
    low = 0.025
    high = 1 - low
    for _ in range(N):
        y = generate_data(sigma2, n)
        cdf = post(y)
        t = cdf(sigma2)
        if low < t and t < high:
            cnt += 1
    return cnt / N

N = 10000
sigmas = [0.1, 0.5, 1.0, 2.0, 5.0]
print('*** constant prior')
np.random.seed(0)
for n in [5, 10, 15, 20]:
    print('** n =', n)
    for i, sigma2 in enumerate(sigmas):
        cov = run_sim(post1, N, sigma2, n)
        print(sigma2, cov)
print('')
print('*** Jeffreys prior')
np.random.seed(0)
for n in [5, 10, 15, 20]:
    print('*** n =', n)
    for i, sigma2 in enumerate(sigmas):
        cov = run_sim(post2, N, sigma2, n)
        print(sigma2, cov)

*** constant prior
** n = 5
0.1 0.9014
0.5 0.9035
1.0 0.9048
2.0 0.9079
5.0 0.9023
** n = 10
0.1 0.9288
0.5 0.9309
1.0 0.9303
2.0 0.9331
5.0 0.9295
** n = 15
0.1 0.9418
0.5 0.9415
1.0 0.9404
2.0 0.9402
5.0 0.9339
** n = 20
0.1 0.9439
0.5 0.9398
1.0 0.9412
2.0 0.9393
5.0 0.9426

*** Jeffreys prior
*** n = 5
0.1 0.9516
0.5 0.9501
1.0 0.9505
2.0 0.948
5.0 0.9506
*** n = 10
0.1 0.9503
0.5 0.949
1.0 0.9511
2.0 0.9514
5.0 0.9497
*** n = 15
0.1 0.9509
0.5 0.952
1.0 0.9513
2.0 0.9501
5.0 0.9486
*** n = 20
0.1 0.9511
0.5 0.948
1.0 0.95
2.0 0.9482
5.0 0.9485


## Example 3: Gaussian With Unknown Mean and Unknown Variance

In [4]:
import numpy as np
from scipy.special import gammaincc

def generate_data(sigma2, n):
    return np.random.normal(0, sigma2**.5, size=n)

def post1(y):
    n = len(y)
    a = np.dot(y, y) - n * np.mean(y)**2
    def f(t):
        if t == 0:
            return 0
        return gammaincc(n/2, a / (2 * t))
    return f

def post2(y):
    n = len(y)
    a = np.dot(y, y) - n * np.mean(y)**2
    def f(t):
        if t == 0:
            return 0
        return gammaincc((n-1)/2, a / (2 * t))
    return f

def run_sim(post, N, sigma2, n):
    cnt = 0
    low = 0.025
    high = 1 - low
    for _ in range(N):
        y = generate_data(sigma2, n)
        cdf = post(y)
        t = cdf(sigma2)
        if low < t and t < high:
            cnt += 1
    return cnt / N

N = 10000
sigmas = [0.1, 0.5, 1.0, 2.0, 5.0]
print('*** Jeffreys prior')
np.random.seed(0)
for n in [5, 10, 15, 20]:
    print('** n =', n)
    for i, sigma2 in enumerate(sigmas):
        cov = run_sim(post1, N, sigma2, n)
        print(sigma2, cov)
print('')
print('*** Reference prior')
np.random.seed(0)
for n in [5, 10, 15, 20]:
    print('** n =', n)
    for i, sigma2 in enumerate(sigmas):
        cov = run_sim(post2, N, sigma2, n)
        print(sigma2, cov)

*** Jeffreys prior
** n = 5
0.1 0.9241
0.5 0.9219
1.0 0.9245
2.0 0.9236
5.0 0.9182
** n = 10
0.1 0.938
0.5 0.9377
1.0 0.94
2.0 0.9391
5.0 0.9395
** n = 15
0.1 0.9419
0.5 0.946
1.0 0.9431
2.0 0.9446
5.0 0.9403
** n = 20
0.1 0.9463
0.5 0.9441
1.0 0.944
2.0 0.9432
5.0 0.9458

*** Reference prior
** n = 5
0.1 0.9533
0.5 0.9528
1.0 0.948
2.0 0.9529
5.0 0.9525
** n = 10
0.1 0.948
0.5 0.9499
1.0 0.9503
2.0 0.9504
5.0 0.9507
** n = 15
0.1 0.9504
0.5 0.9524
1.0 0.9507
2.0 0.9515
5.0 0.9484
** n = 20
0.1 0.9519
0.5 0.9486
1.0 0.9484
2.0 0.9487
5.0 0.9511
